## A simple renewal model
To get started, we'll implement a renewal model
that calculates incidence forward in time
but ignores susceptible depletion and a varying reproduction number,
such that we will consider:
$$I_t = R_0\sum_{\tau<t} I_\tau g_{t-\tau}$$

In [ ]:
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

### Generation time
We'll get a distribution we can sensibly use for the generation time,
which could represent an acute immunising respiratory infection 
(assuming the time unit is days).

In [ ]:
# Generation time summary statistics
gen_mean = 5.0
gen_sd = 1.5

# Calculate equivalent parameters
var = gen_sd ** 2.0
scale = var / gen_mean
a = gen_mean / scale
gamma_params = {"a": a, "scale": scale}

# Get the increment in the CDF
# (i.e. the integral over the increment by one in the distribution)
gen_time_densities = np.diff(gamma.cdf(range(1024), **gamma_params))

pd.Series(gen_time_densities).iloc[0:20].plot(labels={"index": "time", "value": "density"}).update_layout(showlegend=False)

### Calculations
Here, we'll start with naive Python loops with pre-calculated generation times
to be completely explicit (but slow).
Note that the delay is specified as `t - tau - 1` because
delay then starts from zero each time,
which then indexes the first element of the generation time densities.
As shown in the previous cell,
the `gen_time_densities` is the integral of the probability
density over each one-unit interval of the gamma distribution.

In [ ]:
# Let's set some arbitrary parameters to start with
n_times = 25
seed = 1.0
r0 = 2.0
incidence = np.zeros(n_times)
incidence[0] = seed

In [ ]:
for t in range(1, n_times):
    val = 0.0
    for tau in range(t):  # For each day preceding the day of interest
        delay = t - tau - 1  # The generation time index for each preceding day to the day of interest
        val += incidence[tau] * gen_time_densities[delay] * r0  # Calculate the incidence value
    incidence[t] = val

We can get this down to a one-liner if preferred.
The epidemic is going to just keep going up exponentially, of course,
because $R_{0} > 1$ and there is no susceptible depletion.

In [ ]:
alternative_inc = np.zeros(n_times)
alternative_inc[0] = seed

for t in range(1, n_times):
    alternative_inc[t] = (alternative_inc[:t] * gen_time_densities[:t][::-1]).sum() * r0

np.allclose(incidence, alternative_inc) # Check our 2 methods are the same

axis_labels = {"index": "day", "value": "incidence"}
pd.Series(incidence).plot(labels=axis_labels).update_layout(showlegend=False)

Already some interesting phenomena are emerging,
in that the humps are the generations of cases from the first seeding infection
(which occurs at a single time point),
which progressively smooth into one-another with generations of cases.

### Threshold behaviour
We expect a declining epidemic with $R_{0} < 1$, and equilibrium at $R_{0} = 1$

In [ ]:
n_times = 50
low_r_inc = np.zeros(n_times)
low_r_inc[0] = 1.0
# Try changing the r0 value
r0 = 0.9
for t in range(1, n_times):
    low_r_inc[t] = (low_r_inc[:t] * gen_time_densities[:t][::-1]).sum() * r0
pd.Series(low_r_inc).plot(labels=axis_labels).update_layout(showlegend=False)

## Susceptible depletion
To add one layer of realism, we'll now start to think about susceptible depletion,
considering the equation:
$\\I_t = (1-\frac{n_t}{N})R_0\sum_{\tau<t} I_{\tau}g_{t-\tau}$

We'll now run the model with susceptible depletion,
decrementing the susceptible population by the incidence at each step.
We'll also zero out any negative values for the susceptibles
that could occur if the time step is too large
(which should be negligible for reasonable time step and parameter choices).
We'll need a higher reproduction number to deplete
the susceptible population within the time window we have.

In [ ]:
n_times = 30

r0 = 6.0
pop = 100.0
deplete_inc = np.zeros(n_times)
deplete_inc[0] = seed
suscept = pop - seed

for t in range(1, n_times):
    suscept_prop = suscept / pop
    infect_contribution_by_day = deplete_inc[:t] * gen_time_densities[:t][::-1] * r0
    this_inc = infect_contribution_by_day.sum() * suscept_prop
    deplete_inc[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)

pd.Series(deplete_inc).plot(labels=axis_labels).update_layout(showlegend=False)

Now with susceptible depletion, we have an epi-curve that goes up in the initial phase with $R_0 > 1$,
but comes back down as susceptibles are depleted and so $R_t$ falls below one.

## Varying the reproduction number
Building on the previous cells and including susceptible depletion,
we'll now look at varying the reproduction number with time,
because inferring the variation in this quantity is what
we're aiming to achieve from these models.

As previously, the equation we're considering will be:
$\\I_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} I_{\tau}g_{t-\tau}$
However, now the $R_{t}$ value is determined both
by an extrinsic variable ("random") process.
At this stage, the process will be arbitrary values,
and there are several functions that could be used
(including a random walk and an
autoregressive process).

First, let's set up a variable process,
which we'll define as the relative variation in the 
reproduction number as time proceeds.

In [ ]:
n_times = 40
process_req = [2.0, 3.0, 0.1, 5.0]
process_times = np.linspace(0.0, n_times, len(process_req))
process_vals = np.interp(range(n_times), process_times, process_req)
pd.Series(process_vals, index=range(n_times)).plot()

We'll use model parameters, population size
and the generation times as previously, 
and run the model with both susceptible depletion,
and the variable process.
Now we can manipulate the shape of the epicurve.
The evolution of the variable process
is what we'll estimate in later sessions
to estimate the variation in the reproduction number
over time.

In [ ]:
var_r_inc = np.zeros(n_times)
var_r_inc[0] = seed
r0 = 1.0

suscept = pop - seed
for t in range(1, n_times):
    suscept_prop = suscept / pop
    infect_contribution_by_day = var_r_inc[:t] * gen_time_densities[:t][::-1] * r0
    this_inc = infect_contribution_by_day.sum() * suscept_prop * process_vals[t]
    var_r_inc[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)
pd.Series(var_r_inc).plot(labels=axis_labels).update_layout(showlegend=False)